In [ ]:
from kamodo_ccmc.flythrough import SatelliteFlythrough as SF
import datetime as dt

# Set input values for RealFlight function call.
model = 'TIEGCM'
file_dir = 'C:/Users/rringuet/Kamodo_Data/TIEGCM/Uriel_Ramirez_012517_IT_1/'
dataset = 'cnofs'
start_utcts = dt.datetime(2015, 3, 18, 0).replace(tzinfo=dt.timezone.utc).timestamp()
end_utcts = dt.datetime(2015, 3, 21, 0).replace(tzinfo=dt.timezone.utc).timestamp()-1
variable_list = ['T_i'] # list of desired variable names
coord_type = 'GEO'  # GEO cartesian coordinates for the trajectory.
results = SF.RealFlight(dataset, start_utcts, end_utcts, model, file_dir, 
                        variable_list, coord_type)

# Functionalize the results
kamodo_object = SF.WO.Functionalize_SFResults(model,results)
kamodo_object

In [ ]:
# Get CINDI data through HAPI
from kamodo_ccmc.readers.hapi import HAPI

server = 'https://cdaweb.gsfc.nasa.gov/hapi'
dataset = 'CNOFS_CINDI_IVM_500MS'
parameters = 'ionTemperature'
start      = dt.datetime.utcfromtimestamp(start_utcts).isoformat('T')
stop       = dt.datetime.utcfromtimestamp(end_utcts).isoformat('T')
hapiCDA = HAPI(server, dataset, parameters, start, stop)

# Add the new time series into the Kamodo object
kamodo_object = SF.WO.Functionalize_TimeSeries(hapiCDA.tsarray, 'T_iCINDI', 
                                               'K', hapiCDA.ionTemperature(), 
                                               kamodo_object=kamodo_object)
kamodo_object['DIFF'] = 'abs(T_iCINDI - T_i)'  # Analyze.
kamodo_object

In [ ]:
# Now we can plot model and data on the same figure with the difference
kamodo_object.plot('T_i','T_iCINDI','DIFF')